# Training time

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())

import sys
import numpy as np
import json
import os
import sys
import time
import shutil

sys.path.append("/content/drive/My Drive/COLAB/COLAB_SRC/")

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

In [ ]:
import tf_util
import provider

def get_model(point_cloud, input_label, is_training, cat_num, part_num, \
		batch_size, num_point, weight_decay, bn_decay=None, use_bn=True,
		  k = 20, groups = 32, features = 1024, conv_feat = 64):
  """ ConvNet baseline, input is BxNxF point cloud """
  end_points = {}

  batch_size = point_cloud.get_shape()[0].value
  num_point = point_cloud.get_shape()[1].value

  with tf.variable_scope('pro/edge_2') as sc:
      input_image = tf.expand_dims(point_cloud, -1)
      adj = tf_util.pairwise_distance(point_cloud[:,:,:])
      nn_idx, values = knn_range(adj, k=k, search_range=60)
      edge_feature_2 = tf_util.get_edge_feature(input_image, nn_idx=nn_idx, k=k)

  out1 = tf_util.conv2d(edge_feature_2, conv_feat, [1,1],
                       padding='VALID', stride=[1,1],
                       bn=use_bn, is_training=is_training, weight_decay=weight_decay,
                       scope='pro/adj_conv1', bn_decay=bn_decay, activation_fn=tf.nn.elu)
  
  out1 =  tf.contrib.layers.group_norm(out1, groups=groups, scope='pro/gn_1')
  
  with tf.variable_scope('pro/red_1') as sc:
    net_1 = tf.reduce_max(out1, axis=-2, keep_dims=True)

  with tf.variable_scope('pro/edge_3') as sc:
      values_mean_3 = tf.reduce_mean(values[:,:,-1])

      adj = adj + (tf.round(tf.nn.sigmoid(-adj-values_mean_3))*100000000)
      
      nn_idx, values = knn_range(adj, k=k, search_range=60)
      edge_feature_3 = tf_util.get_edge_feature(net_1, nn_idx=nn_idx, k=k)
      
  out3 = tf_util.conv2d(edge_feature_3, conv_feat, [1,1],
                       padding='VALID', stride=[1,1],
                       bn=use_bn, is_training=is_training, weight_decay=weight_decay,
                       scope='pro/adj_conv3', bn_decay=bn_decay, activation_fn=tf.nn.elu)

  out3 =  tf.contrib.layers.group_norm(out3, groups=groups, scope='pro/gn_3')

  with tf.variable_scope('pro/red_2') as sc:
    net_2 = tf.reduce_max(out3, axis=-2, keep_dims=True)

  out7 = tf_util.conv2d(tf.concat([net_1, net_2], axis=-1, name='pro/concat'), 1024, [1, 1], 
                       padding='VALID', stride=[1,1],
                       bn=use_bn, is_training=is_training,
                       scope='pro/adj_conv7', bn_decay=bn_decay, activation_fn=tf.nn.elu)
                       
  out_mean = tf_util.avg_pool2d(out7, [num_point, 1], padding='VALID', scope='pro/meanpool')

  expand = tf.tile(out_mean, [1, num_point, 1, 1], name='pro/expand')
  out7_max = tf_util.conv2d(tf.concat([net_1, net_2, expand], axis=-1, name='pro/concat_max'), 1024, [1, 1], 
                       padding='VALID', stride=[1,1],
                       bn=use_bn, is_training=is_training,
                       scope='pro/adj_conv7_max', bn_decay=bn_decay, activation_fn=tf.nn.elu)
  out_max = tf_util.max_pool2d(out7_max, [num_point, 1], padding='VALID', scope='pro/maxpool')

  pool_concat = tf.concat([out_mean, out_max], axis=-1, name='pro/pool_concat')

  # MLP on global point cloud vector
  net = tf.reshape(pool_concat, [batch_size, -1], name='cla/res')
  
  net = tf_util.fully_connected(net, 512, bn=use_bn, is_training=is_training,
                                scope='cla/fc1', bn_decay=bn_decay, activation_fn=tf.nn.elu)

  net = tf_util.dropout(net, keep_prob=0.5, is_training=is_training,
                         scope='cla/dp1')
                         
  net = tf_util.fully_connected(net, 256, bn=use_bn, is_training=is_training,
                                scope='cla/fc2', bn_decay=bn_decay, activation_fn=tf.nn.elu)

  net = tf_util.dropout(net, keep_prob=0.5, is_training=is_training,
                        scope='cla/dp2')
  net = tf_util.fully_connected(net, cat_num, activation_fn=None, scope='cla/fc3')

  return net, end_points

#########################################


def get_loss(l_pred, label, end_points):
    
    per_instance_label_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=l_pred, labels=label)
    label_loss = tf.reduce_mean(per_instance_label_loss)

    total_loss = label_loss

    return total_loss, label_loss, per_instance_label_loss


def get_loss_weights(l_pred, label, end_points, weights):
    per_instance_label_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=l_pred, labels=label)
    label_loss = tf.reduce_mean(tf.multiply(per_instance_label_loss,weights))
    total_loss = label_loss
    return total_loss, label_loss, per_instance_label_loss

def knn_range(adj_matrix, k=20, search_range=20):
  """Get KNN based on the pairwise distance.
  Args:
    pairwise distance: (batch_size, num_points, num_points)
    k: int
    search_range: int

  Returns:
    nearest neighbors: (batch_size, num_points, k)
  """
  neg_adj = -adj_matrix
  values, nn_idx = tf.nn.top_k(neg_adj, k=search_range)
  nn_idx = tf.transpose(nn_idx)
  nn_idx = tf.random.shuffle(nn_idx)
  nn_idx = tf.transpose(nn_idx)
  return nn_idx[:,:,:k], values

def pairwise_distance(point_cloud):
  """Compute pairwise distance of a point cloud.

  Args:
    point_cloud: tensor (batch_size, num_points, num_dims)

  Returns:
    pairwise distance: (batch_size, num_points, num_points)
  """
  og_batch_size = point_cloud.get_shape().as_list()[0]
  point_cloud = tf.squeeze(point_cloud)
  if og_batch_size == 1:
    point_cloud = tf.expand_dims(point_cloud, 0)
    
  point_cloud_transpose = tf.transpose(point_cloud, perm=[0, 2, 1])
  point_cloud_inner = tf.matmul(point_cloud, point_cloud_transpose)
  point_cloud_inner = -2*point_cloud_inner
  point_cloud_square = tf.reduce_sum(tf.square(point_cloud), axis=-1, keep_dims=True)
  point_cloud_square_tranpose = tf.transpose(point_cloud_square, perm=[0, 2, 1])
  return point_cloud_square + point_cloud_inner + point_cloud_square_tranpose


def knn(adj_matrix, k=20):
  """Get KNN based on the pairwise distance.
  Args:
    pairwise distance: (batch_size, num_points, num_points)
    k: int

  Returns:
    nearest neighbors: (batch_size, num_points, k)
  """
  neg_adj = -adj_matrix
  _, nn_idx = tf.nn.top_k(neg_adj, k=k)
  return nn_idx

def get_edge_feature(point_cloud, nn_idx, k=20):
  """Construct edge feature for each point
  Args:
    point_cloud: (batch_size, num_points, 1, num_dims)
    nn_idx: (batch_size, num_points, k)
    k: int

  Returns:
    edge features: (batch_size, num_points, k, num_dims)
  """
  og_batch_size = point_cloud.get_shape().as_list()[0]
  point_cloud = tf.squeeze(point_cloud)
  if og_batch_size == 1:
    point_cloud = tf.expand_dims(point_cloud, 0)

  point_cloud_central = point_cloud

  point_cloud_shape = point_cloud.get_shape()
  batch_size = point_cloud_shape[0].value
  num_points = point_cloud_shape[1].value
  num_dims = point_cloud_shape[2].value

  idx_ = tf.range(batch_size) * num_points
  idx_ = tf.reshape(idx_, [batch_size, 1, 1]) 

  point_cloud_flat = tf.reshape(point_cloud, [-1, num_dims])
  point_cloud_neighbors = tf.gather(point_cloud_flat, nn_idx+idx_)
  point_cloud_central = tf.expand_dims(point_cloud_central, axis=-2)

  point_cloud_central = tf.tile(point_cloud_central, [1, 1, k, 1])

  edge_feature = tf.concat([point_cloud_central, point_cloud_neighbors-point_cloud_central], axis=-1)
  return edge_feature

In [ ]:
def setup():
  global input_dir, ROTATE, gpu_num, wd, point_num, batch_size, output_dir, all_obj_cats, hdf5_data_dir, hdf5_data_dir_train, color_map, NUM_PART_CATS, NUM_CATEGORIES, TRAINING_FILE_LIST, TESTING_FILE_LIST, TRAINING_EPOCHES, MODEL_STORAGE_PATH, LOG_STORAGE_PATH, SUMMARIES_FOLDER
  
  input_dir = FLAGS_input_dir
 
  ROTATE = bool(FLAGS_rotate)
 
  gpu_num = FLAGS_gpu
 
  wd = FLAGS_wd

  point_num = FLAGS_point_num
  batch_size = 1
  output_dir = FLAGS_output_dir
 
  hdf5_data_dir = os.path.join(BASE_DIR, './' + input_dir)
 
  hdf5_data_dir_train = ''
 
  if not os.path.exists(output_dir):
      os.mkdir(output_dir)
 
  color_map_file = os.path.join(hdf5_data_dir, 'part_color_mapping.json')
  color_map = json.load(open(color_map_file, 'r'))
 
  all_obj_cats_file = os.path.join(hdf5_data_dir, 'all_object_categories.txt')
  fin = open(all_obj_cats_file, 'r')
  lines = [line.rstrip() for line in fin.readlines()]
  all_obj_cats = [(line.split()[0], line.split()[1]) for line in lines]
  fin.close()
 
  all_cats = json.load(open(os.path.join(hdf5_data_dir, 'overallid_to_catid_partid.json'), 'r'))
 
  print( "all_obj_cats_file", all_obj_cats )
 
  NUM_CATEGORIES = len(all_obj_cats)
  NUM_PART_CATS = 1
 
  print('#### Batch Size: {0}'.format(batch_size))
  print('#### Point Number: {0}'.format(point_num))
  print('#### Training using GPU: {0}'.format(gpu_num))
 
 
  TRAINING_EPOCHES = FLAGS_epoch
 
  print('### Training epoch: {0}'.format(TRAINING_EPOCHES))
 
  TRAINING_FILE_LIST = os.path.join(hdf5_data_dir, 'train_hdf5_file_list.txt')
  TESTING_FILE_LIST = os.path.join(hdf5_data_dir, 'val_hdf5_file_list.txt')
 
  MODEL_STORAGE_PATH = os.path.join(output_dir, 'trained_models_o3d')
  if not os.path.exists(MODEL_STORAGE_PATH):
      os.mkdir(MODEL_STORAGE_PATH)
 
  LOG_STORAGE_PATH = os.path.join(output_dir, 'logs')
  if not os.path.exists(LOG_STORAGE_PATH):
      os.mkdir(LOG_STORAGE_PATH)
 
  SUMMARIES_FOLDER =  os.path.join(output_dir, 'summaries')
  if not os.path.exists(SUMMARIES_FOLDER):
      os.mkdir(SUMMARIES_FOLDER) 
 
def printout(flog, data):
    print(data)
    flog.write(data + '\n')
 
def placeholder_inputs():
    pointclouds_ph = tf.placeholder(tf.float32, shape=(batch_size, point_num, 6))
    input_label_ph = tf.placeholder(tf.float32, shape=(batch_size, NUM_CATEGORIES))
    labels_ph = tf.placeholder(tf.int32, shape=(batch_size))
    return pointclouds_ph, input_label_ph, labels_ph
 
 
def convert_label_to_one_hot(labels):
    label_one_hot = np.zeros((labels.shape[0], NUM_CATEGORIES))
    for idx in range(labels.shape[0]):
        label_one_hot[idx, labels[idx]] = 1
    return label_one_hot
 
def get_weights_inverse_num_of_samples(no_of_classes, samples_per_cls, power=1):
    weights_for_samples = 1.0 / np.array(np.power(samples_per_cls, power))
    weights_for_samples = weights_for_samples / np.sum(weights_for_samples) * no_of_classes
    return weights_for_samples
 
def train():
  with tf.Graph().as_default():
        with tf.device('/gpu:'+str(gpu_num)):
            pointclouds_ph, input_label_ph, labels_ph = placeholder_inputs()
            is_training_ph = tf.placeholder(tf.bool, shape=())
 
            min_batch_size = 1
 
            batch = tf.Variable(0, trainable=False)
            learning_rate = tf.train.exponential_decay(
                            BASE_LEARNING_RATE,     # base learning rate
                            batch * min_batch_size,     # global_var indicating the number of steps
                            DECAY_STEP,             # step size
                            DECAY_RATE,             # decay rate
                            staircase=True          # Stair-case or continuous decreasing
                            )
            learning_rate = tf.maximum(learning_rate, LEARNING_RATE_CLIP)
        
            bn_momentum = tf.train.exponential_decay(
                      BN_INIT_DECAY,
                      batch*min_batch_size,
                      BN_DECAY_DECAY_STEP,
                      BN_DECAY_DECAY_RATE,
                      staircase=True)
            bn_decay = tf.minimum(BN_DECAY_CLIP, 1 - bn_momentum)
 
            lr_op = tf.summary.scalar('learning_rate', learning_rate)
            batch_op = tf.summary.scalar('batch_number', batch)
            bn_decay_op = tf.summary.scalar('bn_decay', bn_decay)
 
            with tf.variable_scope(NAMING_SCOPE): 
              labels_pred, end_points = get_model(pointclouds_ph, input_label_ph, \
                      is_training=is_training_ph, bn_decay=bn_decay, cat_num=NUM_CATEGORIES, \
                      part_num=NUM_PART_CATS, batch_size=1, num_point=point_num, weight_decay=wd, use_bn=False)
 
            weights_ph = tf.placeholder(tf.float32, shape=(1))
 
            loss, label_loss, per_instance_label_loss, \
                = get_loss_weights(labels_pred, labels_ph, end_points, weights_ph)
 
            total_training_loss_ph = tf.placeholder(tf.float32, shape=())
            total_testing_loss_ph = tf.placeholder(tf.float32, shape=())
 
            label_training_loss_ph = tf.placeholder(tf.float32, shape=())
            label_testing_loss_ph = tf.placeholder(tf.float32, shape=())
 
 
            label_training_acc_ph = tf.placeholder(tf.float32, shape=())
            label_testing_acc_ph = tf.placeholder(tf.float32, shape=())
 
            total_train_loss_sum_op = tf.summary.scalar('total_training_loss', total_training_loss_ph)
            total_test_loss_sum_op = tf.summary.scalar('total_testing_loss', total_testing_loss_ph)
 
            label_train_loss_sum_op = tf.summary.scalar('label_training_loss', label_training_loss_ph)
            label_test_loss_sum_op = tf.summary.scalar('label_testing_loss', label_testing_loss_ph)
 
 
            label_train_acc_sum_op = tf.summary.scalar('label_training_acc', label_training_acc_ph)
            label_test_acc_sum_op = tf.summary.scalar('label_testing_acc', label_testing_acc_ph)
 
            train_variables = tf.trainable_variables()
 
            trainer = tf.train.AdamOptimizer(learning_rate)
            train_op = trainer.minimize(loss, var_list=train_variables, global_step=batch)
 
            # 0) Retrieve trainable variables
            tvs = tf.trainable_variables() 
            # 1) Create placeholders for the accumulating gradients we'll be storing
            accum_vars = [tf.Variable(tv.initialized_value(),
                          trainable=False) for tv in tvs]
            # 2) Operation to initialize accum_vars to zero
            zero_ops  = [tv.assign(tf.zeros_like(tv)) for tv in accum_vars]
            # 3) Operation to compute the gradients for one minibatch
            gvs = trainer.compute_gradients(loss)
            # 4) Operation to accumulate the gradients in accum_vars
            accum_ops = [accum_vars[i].assign_add(gv[0]) for i, gv in enumerate(gvs)]
            # 5) Operation to perform the update (apply gradients)
            apply_ops = trainer.apply_gradients([(accum_vars[i], tv) for i, tv in enumerate(tf.trainable_variables())], global_step=batch)
            
            
        saver = tf.train.Saver(max_to_keep=5)
 
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = False
        config.allow_soft_placement = True
        sess = tf.Session(config=config)
      
        init = tf.global_variables_initializer()
        sess.run(init)
 
        train_writer = tf.summary.FileWriter(SUMMARIES_FOLDER + '/train', sess.graph)
        test_writer = tf.summary.FileWriter(SUMMARIES_FOLDER + '/test')
 
        train_file_list = provider.getDataFiles(TRAINING_FILE_LIST)
        num_train_file = len(train_file_list)
        test_file_list = provider.getDataFiles(TESTING_FILE_LIST)
        num_test_file = len(test_file_list)

        # write logs to the disk
        if( STORED_MODEL_PATH != '' ):
          flog = open(os.path.join(LOG_STORAGE_PATH, 'log.txt'), 'a')
        else:
          flog = open(os.path.join(LOG_STORAGE_PATH, 'log.txt'), 'w')

        def train_one_epoch(train_file_idx, epoch_num):
            is_training = True
 
            for i in range(num_train_file):
                cur_train_filename = os.path.join(hdf5_data_dir_train, train_file_list[train_file_idx[i]])
                printout(flog, 'Loading train file ' + cur_train_filename)
 
                cur_data, cur_labels = provider.loadDataFile(cur_train_filename)
 
                print( cur_data.shape, cur_labels.shape )
 

                cur_labels = np.squeeze(cur_labels)


                cur_data, cur_labels, order = provider.shuffle_data(cur_data, cur_labels)
                print(cur_data.shape, cur_labels.shape)
 
                weights_for_samples = get_weights_inverse_num_of_samples( NUM_CATEGORIES, [ list(cur_labels).count(i) for i in range(NUM_CATEGORIES)] )
                print("weigths", weights_for_samples)

                cur_labels_one_hot = convert_label_to_one_hot(cur_labels)

                num_data = len(cur_labels)
                num_batch = num_data // min_batch_size
 
                total_loss = 0.0
                total_label_loss = 0.0
                total_label_acc = 0.0
 
                total_guess_class = [0 for _ in range(NUM_CATEGORIES)]
                total_seen_class = [0 for _ in range(NUM_CATEGORIES)]
                total_correct_class = [0 for _ in range(NUM_CATEGORIES)]
 
                print( cur_data.shape )
 
                for j in range(num_batch):
 
                    begidx = j * min_batch_size
                    endidx = (j + 1) * min_batch_size
                    
                    train_data = provider.shuffle_point_cloud(cur_data[begidx: endidx, ...])

 
                    sess.run(zero_ops)
                    for data_min_batch, min_batch in enumerate(range(begidx,endidx)):
                      cur_weights = []
                      for label_itr in cur_labels[min_batch: min_batch+1, ...]:
                        cur_weights.append(weights_for_samples[label_itr])
                      cur_weights = np.array(cur_weights,np.float32)

 

                      feed_dict = {
                              pointclouds_ph: train_data[data_min_batch: data_min_batch+1, :point_num, :],
                              labels_ph: cur_labels[min_batch: min_batch+1, ...], 
                              input_label_ph: cur_labels_one_hot[min_batch: min_batch+1, ...],
                              is_training_ph: is_training, 
                              weights_ph: cur_weights,
                              }
 
                      _, loss_val, label_loss_val, per_instance_label_loss_val, label_pred_val = \
                        sess.run([accum_ops, loss, label_loss, per_instance_label_loss, labels_pred], feed_dict=feed_dict)

 
                      if( np.isnan(loss_val) ):
                        import pdb; pdb.set_trace()
 
                      total_loss += loss_val
                      total_label_loss += label_loss_val
 
                      per_instance_label_pred = np.argmax(label_pred_val, axis=1)
                      total_label_acc += np.mean(np.float32(per_instance_label_pred == cur_labels[min_batch: min_batch+1, ...]))
 
 
                      for i in range(min_batch, min_batch+1):
                        l = cur_labels[i]
                        total_guess_class[ per_instance_label_pred[i-min_batch] ] += 1
                        total_seen_class[l] += 1
                        total_correct_class[l] += (per_instance_label_pred[i-min_batch] == l)
 
                    sess.run(apply_ops)
 
                total_loss = total_loss * 1.0 / num_data
                
                total_label_loss = total_label_loss * 1.0 / num_data
                
                total_label_acc = total_label_acc * 1.0 / num_data
 
                lr_sum, bn_decay_sum, batch_sum, train_loss_sum, train_label_acc_sum, \
                        train_label_loss_sum, \
                        = \
                        sess.run(\
                        [lr_op, bn_decay_op, batch_op, total_train_loss_sum_op, label_train_acc_sum_op, \
                        label_train_loss_sum_op ], \
                        feed_dict={total_training_loss_ph: total_loss, label_training_loss_ph: total_label_loss, \
                        label_training_acc_ph: total_label_acc, \
                        })
 
                train_writer.add_summary(train_loss_sum, i + epoch_num * num_train_file)
                train_writer.add_summary(train_label_loss_sum, i + epoch_num * num_train_file)
             
                
                train_writer.add_summary(lr_sum, i + epoch_num * num_train_file)
                train_writer.add_summary(bn_decay_sum, i + epoch_num * num_train_file)
                train_writer.add_summary(train_label_acc_sum, i + epoch_num * num_train_file)
                
                train_writer.add_summary(batch_sum, i + epoch_num * num_train_file)
 
                learning_rate_show, batch_show, bn_decay_show = sess.run([learning_rate, batch, bn_decay])
 
                printout(flog, '\tLearning Rate: %f' % learning_rate_show)
                printout(flog, '\tbn_decay: %f' % bn_decay_show)
                printout(flog, '\tbatch: %f' % batch_show)
 
                printout(flog, '\tTraining Total Mean_loss: %f' % total_loss)
                printout(flog, '\t\tTraining Label Mean_loss: %f' % total_label_loss)
                printout(flog, '\t\tTraining Label Accuracy: %f' % total_label_acc)
        
                print( "Per instance: ")
                for k in range(NUM_CATEGORIES):
                  print( '\t' + all_obj_cats[k][0], 'XX' if total_guess_class[k] == 0 else total_correct_class[k]/total_guess_class[k], total_correct_class[k]/total_seen_class[k], '---', total_correct_class[k], total_seen_class[k], total_guess_class[k])
 
 
        def eval_one_epoch(epoch_num):
            is_training = False
 
            total_loss = 0.0
            total_label_loss = 0.0
            total_label_acc = 0.0
 
            total_guess_class = [0 for _ in range(NUM_CATEGORIES)]
            total_seen_class = [0 for _ in range(NUM_CATEGORIES)]
            total_correct_class = [0 for _ in range(NUM_CATEGORIES)]
 
            for i in range(num_test_file):
                cur_test_filename = os.path.join(hdf5_data_dir_train, test_file_list[i])
                printout(flog, 'Loading test file ' + cur_test_filename)
 
                print( "cur", cur_test_filename )
 
                cur_data, cur_labels = provider.loadDataFile(cur_test_filename)
                cur_labels = np.squeeze(cur_labels)

                weights_for_samples = get_weights_inverse_num_of_samples( NUM_CATEGORIES, [ list(cur_labels).count(i) for i in range(NUM_CATEGORIES)] )
                print("weigths", weights_for_samples)

                cur_labels_one_hot = convert_label_to_one_hot(cur_labels)
 
                num_data = len(cur_labels)
                num_batch = num_data // batch_size
 
                for j in range(num_batch):
                    begidx = j * batch_size
                    endidx = (j + 1) * batch_size
 
                    cur_weights = []
                    for label_itr in cur_labels[begidx: endidx, ...]:
                      cur_weights.append(weights_for_samples[label_itr])
                    cur_weights = np.array(cur_weights,np.float32)
 
 
                    feed_dict = {
                            pointclouds_ph: cur_data[begidx: endidx, :point_num,:],
                            labels_ph: cur_labels[begidx: endidx, ...], 
                            input_label_ph: cur_labels_one_hot[begidx: endidx, ...], 
                            is_training_ph: is_training, 
                            weights_ph: cur_weights,
                            }
 
                    loss_val, label_loss_val, per_instance_label_loss_val, label_pred_val,\
                            = sess.run([loss, label_loss, per_instance_label_loss, \
                            labels_pred ], \
                            feed_dict=feed_dict)

                    total_loss += loss_val
                    total_label_loss += label_loss_val
 
                    per_instance_label_pred = np.argmax(label_pred_val, axis=1)
                    total_label_acc += np.mean(np.float32(per_instance_label_pred == cur_labels[begidx: endidx, ...]))
                    
                    for i in range(begidx, endidx):
                      total_guess_class[ per_instance_label_pred[i-begidx] ] += 1
                      l = cur_labels[i]
                      total_seen_class[l] += 1
                      total_correct_class[l] += (per_instance_label_pred[i-begidx] == l)
 
            total_loss = total_loss * 1.0 / num_batch            
            total_label_loss = total_label_loss * 1.0 / num_batch            
            total_label_acc = total_label_acc * 1.0 / num_batch
 
            test_loss_sum, test_label_acc_sum, \
                    test_label_loss_sum, \
                    = \
                    sess.run(\
                    [ total_test_loss_sum_op, label_test_acc_sum_op, label_test_loss_sum_op ], \
                    feed_dict={total_testing_loss_ph: total_loss, label_testing_loss_ph: total_label_loss, label_testing_acc_ph: total_label_acc})
 
            test_writer.add_summary(test_loss_sum, i + epoch_num * num_test_file)
            test_writer.add_summary(test_label_loss_sum, i + epoch_num * num_test_file)
            test_writer.add_summary(test_label_acc_sum, i + epoch_num * num_test_file)
 
            printout(flog, '\tTest Total Mean_loss: %f' % total_loss)
            printout(flog, '\t\tTest Label Mean_loss: %f' % total_label_loss)
            printout(flog, '\t\tTest Label Accuracy: %f' % total_label_acc)
 
            mean_pre = 0
            mean_rec = 0
 
            print( "\tPer instance: ")
            for k in range(NUM_CATEGORIES):
                  print( '\t' + all_obj_cats[k][0], 'XX' if total_guess_class[k] == 0 else total_correct_class[k]/total_guess_class[k], total_correct_class[k]/total_seen_class[k], '---', total_correct_class[k], total_seen_class[k], total_guess_class[k])
                  mean_pre += (0 if total_guess_class[k] == 0 else total_correct_class[k]/total_guess_class[k])*total_seen_class[k]
                  mean_rec += (total_correct_class[k]/total_seen_class[k])*total_seen_class[k]
            precision = mean_pre/np.sum(total_seen_class)
            recall = mean_rec/np.sum(total_seen_class)
            print( '\t\tAvg', mean_pre/np.sum(total_seen_class), mean_rec/np.sum(total_seen_class), 2*((precision*recall)/(precision+recall)) )
            print('*****************************')
 
        if not os.path.exists(MODEL_STORAGE_PATH):
            os.mkdir(MODEL_STORAGE_PATH)
 
        for epoch in range(1, TRAINING_EPOCHES + 1):
 
            printout(flog, '\n>>> Training for the epoch %d/%d ...' % (epoch, TRAINING_EPOCHES))
 
            train_file_idx = np.arange(0, len(train_file_list))
            np.random.shuffle(train_file_idx)
 
            seconds = time.time()
 
            train_one_epoch(train_file_idx, epoch)
 
            print('Epoch took %0.3f s' % (time.time()-seconds) )
 
            if( (epoch) % 5 == 0 ):
                printout(flog, '\n<<< Testing on the test dataset ...')
                eval_one_epoch(epoch)
 
            if( (epoch) % 100 == 0 ):
                cp_filename = saver.save(sess, os.path.join(MODEL_STORAGE_PATH, 'epoch_' + str(epoch)+'.ckpt'))
                printout(flog, 'Successfully store the checkpoint model into ' + cp_filename)
 
            flog.flush()
 
        flog.close()

In [ ]:
FLAGS_input_dir = ''
FLAGS_rotate = True

FLAGS_point_num = 32768

FLAGS_batch = 1

FLAGS_gpu = 0
FLAGS_epoch = 200
FLAGS_wd = 0.05

NAMING_SCOPE = "empty"

STORED_MODEL_PATH = "" 

DECAY_STEP = 10000
DECAY_RATE = 0.5
LEARNING_RATE_CLIP = 1e-7
BASE_LEARNING_RATE = 0.0001

BN_INIT_DECAY = 0.5
BN_DECAY_DECAY_RATE = 0.5
BN_DECAY_DECAY_STEP = float(DECAY_STEP * 2)
BN_DECAY_CLIP = 0.99

In [ ]:
# Run left sign precense classification

NAMING_SCOPE = 'cuneiform_left'
BASE_DIR = '/content/drive/My Drive/COLAB/COLAB_DATA/' + NAMING_SCOPE 
shutil.copy2(BASE_DIR+"/cuneiform_file_o3d_left_train.h5", ".")
shutil.copy2(BASE_DIR+"/cuneiform_file_o3d_left_test.h5", ".")
FLAGS_output_dir = BASE_DIR
setup()
train()

In [ ]:
# Run seal precense classification

NAMING_SCOPE = 'cuneiform_seal'
BASE_DIR = '/content/drive/My Drive/COLAB/COLAB_DATA/' + NAMING_SCOPE 
shutil.copy2(BASE_DIR+"/cuneiform_file_o3d_seal_train.h5", ".")
shutil.copy2(BASE_DIR+"/cuneiform_file_o3d_seal_test.h5", ".")
FLAGS_output_dir = BASE_DIR
setup()
train()

In [ ]:
# Run time period comparison with identical dataset

NAMING_SCOPE = 'cuneiform_time_period'
BASE_DIR = '/content/drive/My Drive/COLAB/COLAB_DATA/' + NAMING_SCOPE 
shutil.copy2(BASE_DIR+"/cuneiform_file_o3d_train.h5", ".")
shutil.copy2(BASE_DIR+"/cuneiform_file_o3d_test.h5", ".")
FLAGS_output_dir = BASE_DIR
setup()
train()

In [ ]:
# Run time period comparison with large dataset

NAMING_SCOPE = 'cuneiform_time_period'
BASE_DIR = '/content/drive/My Drive/COLAB/COLAB_DATA/' + NAMING_SCOPE 
shutil.copy2(BASE_DIR+"/cuneiform_file_o3d_train_l.h5", "cuneiform_file_o3d_train.h5")
shutil.copy2(BASE_DIR+"/cuneiform_file_o3d_test.h5", ".")
FLAGS_output_dir = BASE_DIR
setup()
train()